#### Merging the datasets

The script merges the bridge dataset with the GDP data and population density. To do this, two new columns are added to the bridge dataset.
- `GDP`
- `population density`

In [1]:
# load libaries
import pandas as pd
import re

In [2]:
# read data
data_bridges = pd.read_csv('../data/filled_bridge_statistic_germany.csv', sep=';')
data_gdp = pd.read_excel("../data/gdp_dataset.xlsx", sheet_name= "1.1", skiprows= 4)
data_population = pd.read_excel("../data/population_dataset.xlsx", sheet_name= "Kreisfreie Städte u. Landkreise", skiprows= 1)

/opt/miniconda3/envs/MDS/lib/python3.12/site-packages/openpyxl/reader/workbook.py:118: UserWarning: Print area cannot be set to Defined name: Inhalt!$A:$H.
  warn(f"Print area cannot be set to Defined name: {defn.value}.")


In [3]:
data_gdp

,Lfd. Nr.,EU-Code,Regional-schlüssel,Land,NUTS 1,NUTS 2,NUTS 3,Gebietseinheit,1992,1994,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,DE1,8.0,BW,1.0,NaN,NaN,Baden-Württemberg,255866.419,262645.416,...,425366.498,442682.643,463346.012,474915.824,497298.532,516670.072,525421.180,509019.848,539369.060,576128.313
2,2.0,DE11,81.0,BW,NaN,2.0,NaN,"Stuttgart, Regierungsbezirk",110977.071,111602.665,...,180470.227,187384.851,198850.191,201929.136,213043.328,222396.361,221004.577,213383.089,226533.067,242698.643
3,3.0,DE111,8111.0,BW,NaN,NaN,3.0,"Stuttgart, Landeshauptstadt, Stadtkreis",32946.884,31736.568,...,47016.831,48562.097,52459.012,52397.611,54898.140,58227.976,55984.580,53046.137,55421.624,58703.587
4,4.0,DE112,8115.0,BW,NaN,NaN,3.0,"Böblingen, Landkreis",12090.93,11833.816,...,20692.821,21462.358,23986.261,24332.433,25653.016,26077.574,24528.638,23735.777,26455.814,29582.502
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,440.0,DEG0S,16054.0,TH,NaN,NaN,3.0,"Suhl, Kreisfreie Stadt",673.581,916.696,...,1092.421,1135.048,1134.602,1138.482,1166.324,1217.574,1253.990,1168.441,1213.248,1259.931
441,441.0,DEG0T,16070.0,TH,NaN,NaN,3.0,Ilm-Kreis,841.072,1229.313,...,2428.940,2719.023,2836.152,2954.388,3135.141,3144.989,3279.511,3184.624,3212.762,3667.618
442,442.0,DEG0U,16073.0,TH,NaN,NaN,3.0,"Saalfeld-Rudolstadt, Landkreis",1063.227,1623.203,...,2493.720,2544.320,2608.853,2649.259,2736.123,2804.373,2854.321,2728.191,2909.752,3154.812
443,443.0,DEG0V,16072.0,TH,NaN,NaN,3.0,"Sonneberg, Landkreis",448.775,685.658,...,1459.292,1454.498,1480.435,1565.149,1689.768,1679.414,1647.545,1629.570,1643.423,1655.768


Because we want to look at the "Kreise", we deleat the "Bundesländer" that they won't falsify the data. Later we will merge different cities when they are "Kreis" and "Stadt" and deleat the lable to make one "Kreis" out of to simplify it. In addition all other rows which doesnt't contain a velue in the column "Kreis" will be deleted.

In [4]:

# Copy data to modify it
data_gdp_modified = data_gdp.copy()
data_population_modified = data_population.copy()

# Deleating rows without an entery in NUTS 3 (Kreisebene)
data_gdp_modified = data_gdp_modified[
    data_gdp_modified['NUTS 3'].notna() & (data_gdp_modified['NUTS 3'] != '')
]

data_population_modified = data_population_modified[
    data_population_modified['NUTS3'].notna() & (data_population_modified['NUTS3'] != '')
]


Because the names of the GDP dataset amd the population density dataset differ, we normalise the names of the "Kreise"

In [5]:
# normalise names

def normalize_name(name):
    # check if NaN
    if pd.isna(name):
        return name
    # in lowercase letters
    name = name.lower()
    # remove space around hyphen
    name = re.sub(r'\s*-\s*', '-', name)
    # remove commas and full stops
    name = name.replace(',', '')
    name = name.replace('.', '')
    # remove special keywords
    name = re.sub(r'\b(landkreis|kreisfreie stadt|stadt|regierungsbezirk|universitätsstadt|wissenschaftsstadt|landeshauptstadt|reg-bez|region|kfr|hansestadt|stadtkreis|klingenstadt|freie und hansestadt|städteregion|regionalverband|im breisgau)\b', '', name)
    name = re.sub(r'(?<!-)\bkreis\b', '', name) # kreis nur wenn es allein steht
    # umlaute
    # name = re.sub('ä', 'ae', name)
    # name = re.sub('ü', 'ue', name)
    # name = re.sub('ö', 'oe', name)
    # correct spelling
    name = re.sub('ß','ss', name)
    # remove bracket additions
    name = re.sub(r'\([^)]*\)', '', name)
    # remove leading/trailing spaces
    name = name.strip()
    # reduce multiple space character to one
    name = re.sub(r'\s+', ' ', name)
    # special cases
    name = re.sub('ludwigshafen am rhein', 'ludwigshafen', name)
    name = re.sub('neustadt/weinstrasse','neustadt an der weinstrasse', name)
    name = re.sub ('frankfurt main', 'frankfurt am main', name)
    name = re.sub ('hagen der fernuniversität', 'hagen', name)
    name = re.sub ('ostalbkreis', 'ostalb' , name)
    name = re.sub ('neustadt adaisch-bad windsheim', 'neustadt an der aisch-bad windsheim', name)
    name = re.sub ('neustadt adwaldnaab', 'neustadt an der waldnaab', name)
    name = re.sub ('landau in der pfalz', 'Landau', name)
    name = re.sub ('altenburger land', 'altenburg', name)
    name = re.sub ('kassel documenta-', 'kassel', name)
    name = re.sub ('neumarkt idopf', 'neumarkt', name)
    name = re.sub ('offenbach am main', 'offenbach', name)
    name = re.sub ('bremerhaven', 'bremen', name)
    name = re.sub ('weiden idopf', 'weiden', name)
    name = re.sub ('dillingen addonau', 'dillingen', name)
    name = re.sub ('pfaffenhofen adilm', 'pfaffenhofen', name)
    name = re.sub ('mühldorf ainn', 'mühldorf', name)

    return name

df_pop = data_population_modified.copy()
df_gdp = data_gdp_modified.copy()
df_bridges = data_bridges.copy()

# district name normalisation on the datasets
df_pop['Kreisfreie Städte und Landkreise'] = df_pop['Kreisfreie Städte und Landkreise'].apply(normalize_name).str.title()
df_gdp['Gebietseinheit'] = df_gdp['Gebietseinheit'].apply(normalize_name).str.title()
df_bridges['Kreis'] = df_bridges['Kreis'].apply(normalize_name).str.title()


# check whether both dataset contain the same district names
print(f"Districts contained only in population data: {set(df_pop['Kreisfreie Städte und Landkreise']) - set(df_gdp['Gebietseinheit'])}")
print(f"District contained only in GDP data: {set(df_gdp['Gebietseinheit']) - set(df_pop['Kreisfreie Städte und Landkreise'])}")

# delete three rows not corresponding to districts in the population data
df_pop = df_pop[~df_pop['Kreisfreie Städte und Landkreise'].str.contains('mittelfranken', case=False, na=False)]
df_pop = df_pop[~df_pop['Kreisfreie Städte und Landkreise'].str.contains('unterfranken', case=False, na=False)]
df_pop = df_pop[~df_pop['Kreisfreie Städte und Landkreise'].str.contains('schwaben', case=False, na=False)]

# check which districts are only contained in the bridge data
print(f"Districts contained only in bridge data: {set(df_bridges['Kreis']) - set(df_gdp['Gebietseinheit'])}")

# there is one district on the bridge data set (Eisenach) not contained in the other datasets due to the fact that it is an 
# old one, Eisenach nowadays belongs to Wartburgkreis
df_bridges['Kreis'] = data_bridges['Kreis'].replace('Eisenach', 'Wartburgkreis')

# print number of unique districts in each data set
print(f"Number unique districts POPULATION: {len(df_pop['Kreisfreie Städte und Landkreise'].unique())} (total: {len(df_pop)})")
print(f"Number unique districts GDP: {len(df_gdp['Gebietseinheit'].unique())} (total: {len(df_gdp)})")
print(f"Number unique districts BRIDGES: {len(df_bridges['Kreis'].unique())}")

print(f"Districts that are not unique: {df_gdp['Gebietseinheit'].value_counts()[lambda x: x > 1].index.tolist()}")


Districts contained only in population data: {'Schwaben', 'Mittelfranken', 'Unterfranken'}
District contained only in GDP data: set()
Districts contained only in bridge data: {'Eisenach'}
Number unique districts POPULATION: 375 (total: 400)
Number unique districts GDP: 375 (total: 400)
Number unique districts BRIDGES: 375
Districts that are not unique: ['Bayreuth', 'Karlsruhe', 'Regensburg', 'Rostock', 'Bamberg', 'München', 'Coburg', 'Hof', 'Rosenheim', 'Ansbach', 'Fürth', 'Passau', 'Landshut', 'Kassel', 'Würzburg', 'Kaiserslautern', 'Augsburg', 'Aschaffenburg', 'Osnabrück', 'Leipzig', 'Offenbach', 'Bremen', 'Oldenburg', 'Schweinfurt', 'Heilbronn']


Because some cities occur more then once (because we deleted labels like "Kreis" and "Landkreis") we have to merge those rows.

In [6]:
df_pop_norm = df_pop.copy()

# rename columns in pop data set, remove unnecessary columns
print(f"Columns in population data: {df_pop_norm.columns}")
df_pop_norm = df_pop_norm.rename(columns={'NUTS3': 'Code', 'Kreisfreie Städte und Landkreise': 'Kreis', ' Fläche km² ¹⁾': 'Fläche (qkm)', 'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ insgesamt': 'Bevölkerung (insgesamt)', 'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ männlich': 'Bevölkerung (männlich)', 'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ weiblich': 'Bevölkerung (weiblich)', 'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ je km²': 'Bevölkerung (je qkm)'})
df_pop_norm = df_pop_norm.drop(columns=['Amtlicher Regionalschlüssel'])
print(f"Columns in population data: {df_pop_norm.columns}")

# Merge data of cities with the same name due to normalisation. (Kassel Kreis, Kassel Landkreis)
# Merge population
columns_to_sum = [
    'Fläche (qkm)',
    'Bevölkerung (insgesamt)',
    'Bevölkerung (männlich)',
    'Bevölkerung (weiblich)',
    'Bevölkerung (je qkm)'
]

# Group by 'Kreis' and sum all numeric columns
df_pop_cleaned = df_pop_norm.groupby('Kreis', as_index=False)[columns_to_sum].sum()

# save final population dataset
df_pop_cleaned.to_csv('../data/population_dataset_cleaned.csv', sep=',', index=False)

Columns in population data: Index(['Amtlicher Regionalschlüssel', 'Kreisfreie Städte und Landkreise',
       'NUTS3', ' Fläche km² ¹⁾',
       'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ insgesamt',
       'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ männlich',
       'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ weiblich',
       'Bevölkerung auf Grundlage des ZENSUS 2022 ²⁾ je km²'],
      dtype='object')
Columns in population data: Index(['Kreis', 'Code', 'Fläche (qkm)', 'Bevölkerung (insgesamt)',
       'Bevölkerung (männlich)', 'Bevölkerung (weiblich)',
       'Bevölkerung (je qkm)'],
      dtype='object')


To summarize the GDP for those districts, that include a district and a city, we need to know the population of both to be able to calculate the GDP of the combined district. This works as follows: 

GDP_combined_district = (GDP_district * Population_district + GDP_city * Population_city) / (Population_district + Population_city)

Therefore, we need to add the information which row corresponds to the district and which to the city if there are multiple rows with the same district name.

In [7]:
df_gdp_norm = df_gdp.copy()

# rename columns in gdp data set and remove unnecesary ones
#print(f"Columns in GDP data: {df_gdp_norm.columns}")
df_gdp_norm = df_gdp_norm[['Gebietseinheit', 2022, 'EU-Code']]
df_gdp_norm = df_gdp_norm.rename(columns={'Gebietseinheit': 'Kreis', 2022: 'GDP 2022', 'EU-Code': 'Code'})
#print(f"Columns in GDP data: {df_gdp_norm.columns}")

# list all district names occuring twice in the gdp data set
districts = df_gdp_norm['Kreis'].value_counts()[lambda x: x > 1].index.tolist()

# merge GDP
# loop over all districts summarizing multiple districts/cities
merged_dict = dict()
for d_name in districts: 
    gdp = df_gdp_norm.loc[df_gdp_norm['Kreis'] == d_name]
    pop = df_pop_norm.loc[df_pop_norm['Kreis'] == d_name]
    merged = pd.merge(gdp, pop, on='Code')
   
    merged_gdp = (merged['Bevölkerung (insgesamt)'][0] * merged['GDP 2022'][0] + 
                  merged['Bevölkerung (insgesamt)'][1] * merged['GDP 2022'][1]) / (merged['Bevölkerung (insgesamt)'][0] + merged['Bevölkerung (insgesamt)'][1])
    merged_dict[d_name] = merged_gdp

# new GDP
new_df_gdp = pd.DataFrame(columns=['Kreis', 'GDP 2022'])
for i, row in df_gdp_norm.iterrows(): 
    district_name = row['Kreis']
    if district_name in districts: 
        new_df_gdp.loc[len(new_df_gdp)] = [district_name, merged_dict[district_name]]
    else: 
        new_df_gdp.loc[len(new_df_gdp)] = [district_name, df_gdp_norm.loc[df_gdp_norm['Kreis'] == district_name, 'GDP 2022'].iloc[0]]

# remove duplicate rows
df_gdp_cleaned = new_df_gdp.drop_duplicates(subset=['Kreis'])

# save final gdp dataset
df_gdp_cleaned.to_csv('../data/gdp_dataset_cleaned.csv', sep=',', index=False)

Now we merge the corrected bridge dataset with the cleaned population and gdp dataset. 

In [8]:

# merge datasets
gdp_pop = pd.merge(df_gdp_cleaned, df_pop_cleaned[['Kreis', 'Fläche (qkm)', 'Bevölkerung (insgesamt)']], on='Kreis', how='inner')
final_dataset = pd.merge(df_bridges, gdp_pop, on='Kreis', how='inner')

# save final dataset
final_dataset.to_csv('../data/final_bridge_statistic_germany.csv', sep=';', index=False)